# Data Preparation

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
loyalty_scores = pd.read_excel("grocery_database.xlsx", sheet_name ="loyalty_scores" )
customer_details = pd.read_excel("grocery_database.xlsx", sheet_name ="customer_details" )
transactions = pd.read_excel("grocery_database.xlsx", sheet_name ="transactions" )

In [4]:
data_for_reg = pd.merge (customer_details, loyalty_scores, how= 'left', on = 'customer_id')

In [5]:
data_for_reg

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score
0,74,3.38,F,0.59,0.263
1,524,4.76,F,0.52,0.298
2,607,4.45,F,0.49,0.337
3,343,0.91,M,0.54,0.873
4,322,3.02,F,0.63,0.350
...,...,...,...,...,...
865,372,4.38,F,0.50,0.321
866,104,2.36,F,0.63,0.587
867,393,1.87,M,0.59,NaN
868,373,0.21,M,0.47,0.972


In [6]:
sales_summary = transactions.groupby('customer_id').agg({'sales_cost':'sum', 
                                                        'num_items': 'sum',
                                                         'transaction_id':'count',
                                                         'product_area_id': 'nunique' 
                                                         }).reset_index()

In [28]:
sales_summary

,customer_id,sales_cost,num_items,transaction_id,product_area_id
0,1,3980.49,424,51,5
1,2,2056.91,213,52,5
2,3,324.22,65,12,4
3,4,3499.39,278,47,5
4,5,6609.19,987,106,5
...,...,...,...,...,...
865,866,1005.04,226,49,4
866,867,2249.73,244,52,5
867,868,1748.45,329,56,5
868,869,1209.85,184,54,5


In [8]:
sales_summary.columns = ['customer_id', 'total_sales', 'total_items', 'transaction_count', 'product_area_count']

In [9]:
sales_summary

,customer_id,total_sales,total_items,transaction_count,product_area_count
0,1,3980.49,424,51,5
1,2,2056.91,213,52,5
2,3,324.22,65,12,4
3,4,3499.39,278,47,5
4,5,6609.19,987,106,5
...,...,...,...,...,...
865,866,1005.04,226,49,4
866,867,2249.73,244,52,5
867,868,1748.45,329,56,5
868,869,1209.85,184,54,5


In [10]:
sales_summary['average_basket_value'] = sales_summary.total_sales/sales_summary.transaction_count

In [11]:
sales_summary.sample(3)

,customer_id,total_sales,total_items,transaction_count,product_area_count,average_basket_value
371,372,3423.01,338,49,5,69.857347
617,618,418.58,112,12,4,34.881667
7,8,355.54,74,13,4,27.349231


In [12]:
data_for_reg = pd.merge (data_for_reg, sales_summary, how= 'inner', on = 'customer_id')

In [13]:
data_for_reg

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224
3,343,0.91,M,0.54,0.873,2388.31,272,54,5,44.227963
4,322,3.02,F,0.63,0.350,2401.64,278,50,5,48.032800
...,...,...,...,...,...,...,...,...,...,...
865,372,4.38,F,0.50,0.321,3423.01,338,49,5,69.857347
866,104,2.36,F,0.63,0.587,3648.08,280,49,5,74.450612
867,393,1.87,M,0.59,NaN,3067.83,254,53,5,57.883585
868,373,0.21,M,0.47,0.972,3303.80,264,51,5,64.780392


In [14]:
regression_modelling = data_for_reg[data_for_reg.customer_loyalty_score.notna()]
regression_scoring= data_for_reg[data_for_reg.customer_loyalty_score.isna()]

In [15]:
regression_modelling.shape

(400, 10)

In [16]:
regression_scoring.shape

(470, 10)

In [17]:
regression_scoring.head()

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
6,1,4.78,F,0.66,NaN,3980.49,424,51,5,78.048824
7,120,3.49,F,0.38,NaN,2887.20,253,45,5,64.160000
8,52,14.91,F,0.68,NaN,3342.75,335,47,5,71.122340
10,435,0.25,M,0.62,NaN,2326.71,267,48,5,48.473125
12,679,4.74,F,0.58,NaN,3448.59,370,49,5,70.379388


In [18]:
regression_scoring.drop('customer_loyalty_score', axis = 1, inplace = True)

C:\Users\Ibiene\AppData\Local\Temp\ipykernel_27684\865628342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_scoring.drop('customer_loyalty_score', axis = 1, inplace = True)


In [ ]:
pickle.dump(regression_modelling, open("abc_regression_modelling.p", 'wb'))
pickle.dump(regression_scoring, open("abc_regression_scoring.p", 'wb'))

# Modelling Overview

We will build a model that looks to accurately predict the “loyalty_score” metric for those customers that were able to be tagged, based upon the customer metrics listed above.

If that can be achieved, we can use this model to predict the customer loyalty score for the customers that were unable to be tagged by the agency.

As we are predicting a numeric output, we tested three regression modelling approaches, namely:

* Linear Regression
* Decision Tree
* Random Forest

In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import RFECV

In [19]:
data_for_model = pickle.load(open('abc_regression_modelling.p', 'rb'))

In [20]:
data_for_model

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224
3,343,0.91,M,0.54,0.873,2388.31,272,54,5,44.227963
4,322,3.02,F,0.63,0.350,2401.64,278,50,5,48.032800
...,...,...,...,...,...,...,...,...,...,...
862,4,2.14,M,0.64,0.564,3499.39,278,47,5,74.455106
865,372,4.38,F,0.50,0.321,3423.01,338,49,5,69.857347
866,104,2.36,F,0.63,0.587,3648.08,280,49,5,74.450612
868,373,0.21,M,0.47,0.972,3303.80,264,51,5,64.780392


In [21]:
data_for_model.drop('customer_id', axis = 1, inplace= True)

In [23]:
# Shuffle data, best practice
data_for_model= shuffle(data_for_model, random_state=42)


In [24]:
# Check for missing values
print(data_for_model.isna().sum()/len(data_for_model))
data_for_model.shape

distance_from_store       0.0050
gender                    0.0075
credit_score              0.0050
customer_loyalty_score    0.0000
total_sales               0.0000
total_items               0.0000
transaction_count         0.0000
product_area_count        0.0000
average_basket_value      0.0000
dtype: float64


(400, 9)

In [25]:
# Drop nulls since % is small
data_for_model.dropna(how = 'any', inplace=True)


In [26]:
data_for_model.shape

(394, 9)

In [28]:
# Outlier Investigation

outlier_investigation = data_for_model.describe()
outlier_investigation


,distance_from_store,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
count,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,2.021041,0.596117,0.509589,1846.502538,278.304569,44.926396,4.312183,36.781487
std,2.566045,0.101352,0.294504,1767.829482,214.236424,21.248012,0.725054,19.337336
min,0.000000,0.260000,0.002000,45.950000,10.000000,4.000000,2.000000,9.342000
25%,0.710000,0.530000,0.263000,942.067500,201.000000,41.000000,4.000000,22.406953
50%,1.645000,0.590000,0.510500,1471.490000,258.500000,50.000000,4.000000,30.365417
75%,2.910000,0.660000,0.771500,2104.727500,318.500000,53.000000,5.000000,47.208474
max,44.370000,0.880000,1.000000,9878.760000,1187.000000,109.000000,5.000000,102.335417


In [32]:
outlier_columns = ['distance_from_store', 'total_sales', 'total_items']

for column in outlier_columns:
    lower_quartile = data_for_model[column].quantile(0.25)
    upper_quartile = data_for_model[column].quantile(0.75)
    iqr = upper_quartile - lower_quartile
    iqr_extended = iqr * 2
    min_border = lower_quartile - iqr_extended
    max_border = upper_quartile + iqr_extended

    outliers = data_for_model[(data_for_model[column] < min_border) |(data_for_model[column] >max_border)].index
    print(f"{len(outliers)} outliers detected in column {column}")

    data_for_model.drop(outliers, inplace = True)

2 outliers detected in column distance_from_store
23 outliers detected in column total_sales
0 outliers detected in column total_items


In [33]:
data_for_model.shape

(369, 9)

In [45]:
# split data into X and y 
X = data_for_model.drop('customer_loyalty_score', axis =1 )
y = data_for_model['customer_loyalty_score']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [47]:
# Deal with categorical variables
categorical_vars = ['gender']
one_hot_encoder = OneHotEncoder(sparse = False, drop = 'first')
X_train_encoded = one_hot_encoder.fit_transform(X_train[categorical_vars])
X_test_encoded = one_hot_encoder.transform(X_test[categorical_vars])

encoder_feature_names = one_hot_encoder.get_feature_names(categorical_vars)

X_train_encoded = pd.DataFrame(X_train_encoded, columns = encoder_feature_names)
X_train = pd.concat([X_train.reset_index(drop = True),X_train_encoded.reset_index(drop = True)], axis = 1)
X_train.drop(categorical_vars, axis = 1, inplace = True)



In [48]:
X_test_encoded = pd.DataFrame(X_test_encoded, columns = encoder_feature_names)
X_test = pd.concat([X_test.reset_index(drop = True),X_test_encoded.reset_index(drop = True)], axis = 1)
X_test.drop(categorical_vars, axis = 1, inplace = True)

In [49]:
X_train.head()

,distance_from_store,credit_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value,gender_M
0,0.82,0.62,1008.62,228,51,4,19.776863,0.0
1,0.40,0.56,1290.81,226,54,5,23.903889,1.0
2,0.06,0.27,1252.39,219,55,5,22.770727,1.0
3,3.42,0.62,2422.70,224,51,5,47.503922,0.0
4,1.51,0.59,1208.91,198,55,5,21.980182,0.0


In [64]:
# Feature selection 

regressor = LinearRegression()
feature_selector = RFECV(regressor)

fit = feature_selector.fit(X_train, y_train)
optimal_feature_count = feature_selector.n_features_
print(f"Optimal number of features: {optimal_feature_count}")

X_train= X_train.loc[:, feature_selector.get_support()]
X_test = X_test.loc[:, feature_selector.get_support()]


fig = px.scatter(x =range(1, len(fit.grid_scores_)+1), y =fit.grid_scores_, title = 'feature selction using RFECV')
fig.show()

Optimal number of features: 8


In [65]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [68]:
y_pred = regressor.predict(X_test)

In [75]:
r_squared = r2_score(y_test, y_pred)
print(r_squared)

0.8058853611654755


In [72]:
#Cross Validation
cv = KFold(n_splits = 4, shuffle = True, random_state= 42)
cv_scores = cross_val_score(regressor, X_train, y_train, cv = cv, scoring = 'r2')
cv_scores.mean()


0.8567108687177778

In [76]:
# Adjusted R squared 

num_data_points , num_input_vars = X_test.shape
num_data_points, num_input_vars

adjusted_r_sqaured = 1 - (1-r_squared)*(num_data_points -1 )/(num_data_points-num_input_vars-1)

In [77]:
print(adjusted_r_sqaured)

0.781994328693534

In [81]:
coefficients = pd.DataFrame(regressor.coef_)
input_var_name = pd.DataFrame(X_train.columns)


In [90]:
summary_stats = pd.concat([input_var_name, coefficients], axis =1)
summary_stats.columns = ['input_vars', 'coefficient']
summary_stats.sort_values(by = 'coefficient',  ascending= False)

,input_vars,coefficient
5,product_area_count,0.062870
7,gender_M,0.028032
3,total_items,0.001067
2,total_sales,0.000145
6,average_basket_value,-0.004495
4,transaction_count,-0.004879
1,credit_score,-0.041398
0,distance_from_store,-0.185551


In [91]:
regressor.intercept_

0.4761753877922798